In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
import os
from scipy import signal
from scipy.signal import savgol_filter
from scipy.signal import lfilter
from statistics import mean

#first test
waterflowData = pd.read_csv(os.getcwd() + "/data/WaterFlowTest_08_08_16_42")
tankData = waterflowData['low2'].to_numpy()
amountOfData = len(tankData)
currPressureData = np.asarray([0] * amountOfData)

#second test
waterflowData2 = pd.read_csv(os.getcwd() + "/data/waterflow_test_full_9_5_16_16")
tankData2 = waterflowData2['low1'].to_numpy()
amountOfData2 = len(tankData2)
currPressureData2 = np.asarray([0] * amountOfData2)

#third test
waterflowData3 = pd.read_csv(os.getcwd() + "/data/waterflow_test_full_9_5_14_31 (1)")
tankData3 = waterflowData3['low1'].to_numpy()
amountOfData3 = len(tankData3)
currPressureData3 = np.asarray([0] * amountOfData3)

In [2]:
#sixth test
waterflowData6 = pd.read_csv(os.getcwd() + "/data/waterflow_09-26-20__15_17 - waterflow_09-26-20__15_17.txt")
tankData6 = waterflowData6["time"].to_numpy()
time=[]
high=[]
lox_tank=[]
propane_tank=[]
lox_injector=[]
propane_injector=[]
for x in range(len(tankData6)):
    breakUp=tankData6[x].split()
    time.append(int(breakUp[0]))
    high.append(int(breakUp[1]))
    lox_tank.append(int(breakUp[2]))
    propane_tank.append(int(breakUp[3]))
    lox_injector.append(int(breakUp[4]))
    propane_injector.append(int(breakUp[5]))

In [9]:
##current version of detect function
##added third "zoom-in" without dary data 
def detectPeaks(data):
    xaxis = np.arange(0,len(data),1)
    print("Amount of Data Points:", len(data))
    n = 100  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filter_data2 = lfilter(b,a,data)
    dary = np.array([*map(float, data)])
    dary -=np.average(dary)
    step = np.hstack((np.ones(len(dary)), -1*np.ones(len(dary))))

    dary_step = np.convolve(dary, step, mode='valid')
    peaksall=[]
    peaks = signal.find_peaks(dary_step, width=20)[0]
    print("trying it out:", peaks)
    if len(peaks)>0:
        for p in peaks:
            
            peaksall.append(np.array([p]))
    print("Positive Peaks:", len(peaks))
    peaks2 = signal.find_peaks(-dary_step, width=20)[0]
    print("Negative Peaks:", len(peaks2))
    if len(peaks2)>0:
        for p in peaks2:
            
            peaksall.append(np.array([p]))
    print("Total Peaks detected:", len(peaksall))
    print(peaksall)
    plt.figure()

    plt.plot(dary)

    plt.plot(dary_step/10)

    for ii in range(len(peaks)):
        plt.plot((peaks[ii], peaks[ii]), (-1500, 1500), 'r')

    plt.show()
    ##start of second round
    #creating new data set with first and last peak as lower, upper bounds
    if len(peaksall)>=2:
        data2 = data[int(peaksall[0][0]):int(peaksall[-1][-1])]
    else:
        data2 = data[0:int(peaksall[-1])]
    ##
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filter_data = lfilter(b,a,data2)
    ##
    dary2 = np.array([*map(float, data2)])
    
    dary2 -=np.average(dary2)
    step2 = np.hstack((np.ones(len(dary2)), -1*np.ones(len(dary2))))

    dary_step2 = np.convolve(dary2, step2, mode='valid')

    # Get the peaks of the convolution
    #negative peaks
    peaks3 = signal.find_peaks(-dary_step2, width=20)[0]
    #positive peaks
    peaks4 = signal.find_peaks(dary_step2, width=20)[0]
    #adjusting for frame shift
    if len(peaksall)>=2:
        if len(peaks3)>0:
            for p in range(len(peaks3)):
                peaksall.append(np.array([peaks3[p]+peaks[0]+1]))
            #peaksall.append(peaks3+peaks+1)
        print("Negative Peaks Detected:", len(peaks3), "at", peaks3)
        if len(peaks4)>0:
            for p in range(len(peaks4)):
                peaksall.append(np.array([peaks4[p]+peaks[0]+1]))
            #peaksall.append(peaks4+peaks+1)
        print("Positive Peaks Detected:", len(peaks4), "at", peaks4)
        print(peaksall)
    else:
        if len(peaks3)>0:
            for p in range(len(peaks3)):
                peaksall.append(np.array([peaks3[p]]))
        print("Negative Peaks Detected:", len(peaks3),"at",peaks3)
        if len(peaks4)>0:
            for p in peaks4:
                
                peaksall.append(np.array([p]))
        print("Positive Peaks Detected:", len(peaks4),"at", peaks4)
        print(peaksall)
    ###make this more robust
    #peaks2 = signal.find_peaks(-dary_step, width=20)[0]
    print(peaks3)

    # plots
    plt.figure()

    plt.plot(dary2)
    #orange:
    plt.plot(dary_step2/10)
    #repeat process on dary data:
    n = 20  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filter_data3 = lfilter(b,a,dary_step2/10)
    peaks40 = signal.find_peaks(filter_data3, width=20)[0]
    peaks30 = signal.find_peaks(-filter_data3, width=20)[0]
    peaksall2=[]
    plt.plot(filter_data3)
    for ii in range(len(peaks40)):
        plt.plot((peaks40[ii], peaks40[ii]), (-150, 150), 'green')
    ##adding to master graph
    if len(peaksall)>=2:
        if len(peaks30)>0:
            for p in range(len(peaks30)):
                print("testing peaks:",peaks)
                peaksall2.append(np.array([peaks30[p]+peaks[p]+1]))
                print("testing something:", np.array([peaks30[p]+peaks[p]+1]))
            #peaksall2.append(peaks30+peaks+1)
        print("Negative Peaks Detected:", len(peaks30), "at", peaks30)
        if len(peaks40)>0:
            for p in range(len(peaks40)):
                peaksall2.append(np.array([peaks40[p]+peaks[p]+1]))
            #peaksall2.append(peaks40+peaks+1)
        print("Positive Peaks Detected:", len(peaks40), "at", peaks40)
        print("This is peaksall2:",peaksall2)
    else:
        if len(peaks30)>0:
            for p in peaks30:
                peaksall2.append(np.array([p]))
            #peaksall2.append(peaks30)
        print("Negative Peaks Detected:", len(peaks30),"at",peaks30)
        if len(peaks40)>0:
            for p in peaks40:
                peaksall.append(np.array([p]))
            #peaksall2.append(peaks40)
        print("Positive Peaks Detected:", len(peaks40),"at", peaks40)
        print(peaksall2)
    ## appending 2nd round of lines to first
    
    peaksall = peaksall + peaksall2
    print("This is peaksall:", peaksall)
    peaksall.sort()
    
        
    for ii in range(len(peaks3)):
        plt.plot((peaks3[ii], peaks3[ii]), (-150, 150), 'r')

    plt.show()
    
    ##end of second round
    print("end of second round")
    
    ##start of third round
    #creating new data set with first and last peak as lower, upper bounds
    if len(peaksall)>=2:
        data2 = data[int(peaksall[0][0]):int(peaksall[2][-1])]
    else:
        data2 = data[0:int(peaksall[-1])]
    ##
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filter_data4 = lfilter(b,a,data2)
    ##
    dary2 = np.array([*map(float, filter_data4)])
    
    dary2 -=np.average(dary2)
    step2 = np.hstack((np.ones(len(dary2)), -1*np.ones(len(dary2))))

    dary_step2 = np.convolve(dary2, step2, mode='valid')

    # Get the peaks of the convolution
    #negative peaks
    peaks3 = signal.find_peaks(-dary_step2, width=20)[0]
    #positive peaks
    peaks4 = signal.find_peaks(dary_step2, width=20)[0]
    #adjusting for frame shift
    if len(peaksall)>=2:
        if len(peaks3)>0:
            for p in range(len(peaks3)):
                peaksall.append(np.array([peaks3[p]+peaks[0]+1]))
            #peaksall.append(peaks3+peaks+1)
        print("Negative Peaks Detected:", len(peaks3), "at", peaks3)
        if len(peaks4)>0:
            for p in range(len(peaks4)):
                peaksall.append(np.array([peaks4[p]+peaks[0]+1]))
            #peaksall.append(peaks4+peaks+1)
        print("Positive Peaks Detected:", len(peaks4), "at", peaks4)
        print(peaksall)
    else:
        if len(peaks3)>0:
            for p in range(len(peaks3)):
                peaksall.append(np.array([peaks3[p]]))
        print("Negative Peaks Detected:", len(peaks3),"at",peaks3)
        if len(peaks4)>0:
            for p in peaks4:
                
                peaksall.append(np.array([p]))
        print("Positive Peaks Detected:", len(peaks4),"at", peaks4)
        print(peaksall)
    ###make this more robust
    #peaks2 = signal.find_peaks(-dary_step, width=20)[0]
    print(peaks3)

    # plots
    plt.figure()

    plt.plot(dary2)
    #orange:
    plt.plot(dary_step2/10)
    
    peaksall.sort()
    
        
    for ii in range(len(peaks3)):
        plt.plot((peaks3[ii], peaks3[ii]), (-150, 150), 'r')

    plt.show()
    
    ##end of third round
    print("end of third round")
    

    plt.figure(dpi=100, figsize=(20,5))
    plt.plot(xaxis, data)
    
    print(peaksall)
    for i in peaksall:
        for x in i:
            plt.axvline(x,c='k', lw='1')
    
    for i in peaksall2:
        for x in i:
            plt.axvline(x,c='green', lw='1')
    plt.show()
    
    ###removing lines from user input
    user_input = input("List of lines to remove, separated by comma (starting at index 0), e.g. 0,2,5: ")
    
    if user_input:
        a_list =  list(map(int,user_input.split(',')))
        print(a_list)
        print(peaksall)

        #removing lines at index indicated by user input, using inserts to maintain list length in process
        for i in a_list:
            del peaksall[i]
            peaksall.insert(i,np.array([69.69]))
        print("dfahkdfakhdf", peaksall)

        #removing the replaced items in peaksall
        peaksall = [x for x in peaksall if x!=([69.69])]
        #peaksall = [x  for x in range(len(peaksall)) if not peaksall[a_list[x]]]

        plt.figure(dpi=100, figsize=(20,5))
        plt.plot(xaxis, data)

        print(peaksall)
        for i in peaksall:
            for x in i:
                plt.axvline(x,c='k', lw='1')


        plt.show()
        
    ###shifting lines with user input
    
    user_input2 = input("List of lines to edit, separated by comma (starting at index 0), e.g. 0,2,5: ") 
    
    if user_input2:
        b_list =  list(map(int,user_input2.split(',')))
    
    for i in b_list:
        
        print("Y axis value", "Corresponding Pressure")
        print(peaksall[i]-5, data[int(peaksall[i]-5)])
        print(peaksall[i]-4, data[int(peaksall[i]-4)])
        print(peaksall[i]-3, data[int(peaksall[i]-3)])
        print(peaksall[i]-2, data[int(peaksall[i]-2)])
        print(peaksall[i]-1, data[int(peaksall[i]-1)])
        print("Current Line:")
        print(peaksall[i], data[int(peaksall[i])])
        print(peaksall[i]+1, data[int(peaksall[i]+1)])
        print(peaksall[i]+2, data[int(peaksall[i]+2)])
        print(peaksall[i]+3, data[int(peaksall[i]+3)])
        print(peaksall[i]+4, data[int(peaksall[i]+4)])
        print(peaksall[i]+5, data[int(peaksall[i]+5)])
        
        user_input3 = input("Choose X value for new line or type 0 to delete")
        
        del peaksall[i]
        if user_input3!=0:
        
            peaksall.insert(i, np.array([int(user_input3)]))
    
    
    
    plt.figure(dpi=100, figsize=(20,5))
    plt.plot(xaxis, data)

    print(peaksall)
    for i in peaksall:
        for x in i:
            plt.axvline(x,c='k', lw='1')


    plt.show()    
    
    ##taking input on data type and returning parameters according to data type
    ##can probably be done with radios in gui
    data_type = input("Data Type (high, propane injector, propane tank, lox injector, or lox tank)")
    
    if data_type=="propane tank" or "lox tank":
        print("propane_tank")
        static_condition = data[int(peaksall[0]):int(peaksall[1])]
        static_pressure = mean(static_condition)
        print("Static Pressure:", static_pressure)
        dynamic_condition = data[int(peaksall[1]):int(peaksall[2])]
        dynamic_pressure = mean(dynamic_condition)
        print("Dynamic Pressure:", dynamic_pressure)
        droop = static_pressure - dynamic_pressure 
        print("Droop:", droop)
        emptying_time = int(peaksall[2])-int(peaksall[1])
        print("Emptying Time:", emptying_time)
        dynamic_rate_of_increase = (data[int(peaksall[2])]-data[int(peaksall[1])])/emptying_time
        print("Dyanmic Rate of Increase:", dynamic_rate_of_increase)
    elif data_type=="propane injector" or "lox injector":
        print("propane_injector")
        
    
    elif data_type=="high":
        print("high")
    else:
        ##implement a better way to do this
        print("try again")
        
        
        
        
        
        
        
    
   

In [ ]:
detectPeaks(tankData)

In [ ]:
detectPeaks(propane_tank)